This notebook contatins application of neural network described in https://arxiv.org/abs/1611.08024 to the proposed dataset.

In [1]:
%load_ext autoreload
%autoreload 2

In [26]:
from src.data import DataBuildClassifier
import shutil
import os
import numpy as np
from sklearn.model_selection import train_test_split
from src.utils import single_auc_loging
from src.utils import prepare_dirs,write_results_table, separte_last_block
from src.model_torch import train_model_eegnet
import shutil
from sklearn.model_selection import StratifiedKFold


In [6]:
experiment_res_dir = './res/' #Path to save results and training|testing statistics
all_subjects = [25,26,27,28,29,30,32,33,34,35,36,37,38]
data = DataBuildClassifier('/home/likan_blk/BCI/NewData')

Hyperparameters of the pipeline (EEGNET mostly)

In [7]:
params = {'resample_to': 369,
                 'D': 3,
                 'F1': 12,
                 'dropoutRate1': 0.52,
                 'dropoutRate2': 0.36,
                 'lr': 0.00066,
                 'norm_rate': 0.275
                 }

In [8]:
subjects = data.get_data(all_subjects,shuffle=False, windows=[(0.2,0.5)],baseline_window=(0.2,0.3),resample_to=params['resample_to'])

Following block contains a defenition of crossvalidation function and loop, where model created and consequentially applied for each subjects data and per-subject training information saved to corresponding folders in experiment_res_dir. Please look carefully to cv_per_subj_test function.

In [22]:
def cv_per_subj_test(x,y,params,path_to_subj, test_on_last_block=False, plot_fold_history=False):
    model_path = os.path.join(path_to_subj,'checkpoints')
    best_val_epochs = []
    best_val_aucs = []
    folds = 4  # To preserve split as 0.6 0.2 0.2
    if test_on_last_block:
        x_tr,y_tr,x_tst,y_tst = separte_last_block(x,y,test_size=0.2)

    cv = StratifiedKFold(n_splits=folds, shuffle=True)
    cv_splits = list(cv.split(x_tr, y_tr))
    for fold, (train_idx, val_idx) in enumerate(cv_splits):
        fold_model_path = os.path.join(model_path, '%d' % fold)
        os.makedirs(fold_model_path)
        x_tr_fold, y_tr_fold = x_tr[train_idx], y_tr[train_idx]
        x_val_fold, y_val_fold = x_tr[val_idx], y_tr[val_idx]
        val_history, fold_model = train_model_eegnet(x_tr_fold,y_tr_fold,params,(x_val_fold,y_val_fold),epochs=200,
                                                     batch_size=32, shuffle=True,
                                                     model_path=os.path.join(fold_model_path,'model{}'.format(fold)))
        best_val_epochs.append(np.argmax(val_history['val_auc']) + 1)  # epochs count from 1 (not from 0)
        best_val_aucs.append(np.max(val_history['val_auc']))
        if plot_fold_history:
            single_auc_loging(val_history, 'fold %d' % fold, fold_model_path)

    if test_on_last_block:
        test_history, final_model = train_model_eegnet(x_tr, y_tr, params, epochs=int(np.mean(best_val_epochs)),
                                                       validation_data=(x_tst, y_tst), batch_size=32, shuffle=True,
                                                       model_path=os.path.join(path_to_subj,'naive_model'))

    single_auc_loging(test_history, 'test_history', path_to_save=path_to_subj)
    with codecs.open('%s/res.txt' % path_to_subj, 'w', encoding='utf8') as f:
        f.write(u'Val auc %.02f±%.02f\n' % (np.mean(best_val_aucs),np.std(best_val_aucs)))
        f.write('Test auc naive %.02f\n' % (test_history['val_auc'][-1]))

    return {'val_auc':test_history['val_auc'][-1]}, final_model

In [24]:
experiment_res_dir = './res/'
subjs_test_stats = {}
for train_subject in all_subjects:
    path_to_subj = prepare_dirs(experiment_res_dir, train_subject)
    x = subjects[train_subject][0]
    x = x.transpose(0, 2, 1)[:, np.newaxis, :, :]
    y=subjects[train_subject][1]
    test_stats, model = cv_per_subj_test(x, y, params, path_to_subj,test_on_last_block=True, plot_fold_history=True)
    subjs_test_stats[train_subject] = test_stats

Epoch %d: train loss %f 0 0.6552248341696603
Epoch 0: val loss 0.660504

Epoch %d: train loss %f 1 0.6497832706996373
Epoch 1: val loss 0.655750

Epoch %d: train loss %f 2 0.626430332660675
Epoch 2: val loss 0.655168

Epoch %d: train loss %f 3 0.601343435900552
Epoch 3: val loss 0.650789

Epoch %d: train loss %f 4 0.6161148122378758
Epoch 4: val loss 0.642168

Epoch %d: train loss %f 5 0.5858955809048244
Epoch 5: val loss 0.632311

Epoch %d: train loss %f 6 0.5581608840397426
Epoch 6: val loss 0.629444

Epoch %d: train loss %f 7 0.578955956867763
Epoch 7: val loss 0.631932

Epoch %d: train loss %f 8 0.5981648862361908
Epoch 8: val loss 0.626484

Epoch %d: train loss %f 9 0.5488475518567222
Epoch 9: val loss 0.615948

Epoch %d: train loss %f 10 0.5561829379626683
Epoch 10: val loss 0.612160

Epoch %d: train loss %f 11 0.5746537957872663
Epoch 11: val loss 0.606759

Epoch %d: train loss %f 12 0.6262703112193516
Epoch 12: val loss 0.607629

Epoch %d: train loss %f 13 0.5555882155895233
Ep

Epoch %d: train loss %f 112 0.3562554589339665
Epoch 112: val loss 0.787152

Epoch %d: train loss %f 113 0.3567582837172917
Epoch 113: val loss 0.795499

Epoch %d: train loss %f 114 0.3814563921519688
Epoch 114: val loss 0.820785

Epoch %d: train loss %f 115 0.397169543164117
Epoch 115: val loss 0.899031

Epoch %d: train loss %f 116 0.34944408919130054
Epoch 116: val loss 0.885589

Epoch %d: train loss %f 117 0.3574463427066803
Epoch 117: val loss 0.794445

Epoch %d: train loss %f 118 0.35440058367592947
Epoch 118: val loss 0.770798

Epoch %d: train loss %f 119 0.32065450080803465
Epoch 119: val loss 0.807230

Epoch %d: train loss %f 120 0.3672822820288794
Epoch 120: val loss 0.818173

Epoch %d: train loss %f 121 0.3279714754649571
Epoch 121: val loss 0.824003

Epoch %d: train loss %f 122 0.34095323937279837
Epoch 122: val loss 0.841497

Epoch %d: train loss %f 123 0.30651891018663135
Epoch 123: val loss 0.889857

Epoch %d: train loss %f 124 0.32251579633780886
Epoch 124: val loss 0.88

Epoch %d: train loss %f 22 0.5121886347021375
Epoch 22: val loss 0.577586

Epoch %d: train loss %f 23 0.5420637003013066
Epoch 23: val loss 0.579071

Epoch %d: train loss %f 24 0.5415372805935996
Epoch 24: val loss 0.585995

Epoch %d: train loss %f 25 0.6502721394811358
Epoch 25: val loss 0.592663

Epoch %d: train loss %f 26 0.5311219819954464
Epoch 26: val loss 0.589024

Epoch %d: train loss %f 27 0.5170711406639644
Epoch 27: val loss 0.574578

Epoch %d: train loss %f 28 0.5407811147826058
Epoch 28: val loss 0.559721

Epoch %d: train loss %f 29 0.49454098514148165
Epoch 29: val loss 0.561462

Epoch %d: train loss %f 30 0.510971086365836
Epoch 30: val loss 0.565231

Epoch %d: train loss %f 31 0.457245762859072
Epoch 31: val loss 0.576387

Epoch %d: train loss %f 32 0.4857593093599592
Epoch 32: val loss 0.577639

Epoch %d: train loss %f 33 0.46397125720977783
Epoch 33: val loss 0.587048

Epoch %d: train loss %f 34 0.47372277293886456
Epoch 34: val loss 0.596495

Epoch %d: train loss %f 

Epoch %d: train loss %f 135 0.30151085768427166
Epoch 135: val loss 0.750117

Epoch %d: train loss %f 136 0.29968592311654774
Epoch 136: val loss 0.760864

Epoch %d: train loss %f 137 0.35354551672935486
Epoch 137: val loss 0.785406

Epoch %d: train loss %f 138 0.27545122376510073
Epoch 138: val loss 0.808741

Epoch %d: train loss %f 139 0.3793928921222687
Epoch 139: val loss 0.798691

Epoch %d: train loss %f 140 0.28313964179583956
Epoch 140: val loss 0.796881

Epoch %d: train loss %f 141 0.26951934822968077
Epoch 141: val loss 0.779568

Epoch %d: train loss %f 142 0.3918942255633218
Epoch 142: val loss 0.796267

Epoch %d: train loss %f 143 0.3243300552879061
Epoch 143: val loss 0.844159

Epoch %d: train loss %f 144 0.2816403742347445
Epoch 144: val loss 0.717253

Epoch %d: train loss %f 145 0.29528051401887623
Epoch 145: val loss 0.730919

Epoch %d: train loss %f 146 0.313095360994339
Epoch 146: val loss 0.753514

Epoch %d: train loss %f 147 0.30330207092421396
Epoch 147: val loss 0.

Epoch %d: train loss %f 45 0.43926974705287386
Epoch 45: val loss 0.669826

Epoch %d: train loss %f 46 0.38885108062199186
Epoch 46: val loss 0.698737

Epoch %d: train loss %f 47 0.39665113176618305
Epoch 47: val loss 0.718566

Epoch %d: train loss %f 48 0.39271639926092966
Epoch 48: val loss 0.728563

Epoch %d: train loss %f 49 0.3956956905978067
Epoch 49: val loss 0.719102

Epoch %d: train loss %f 50 0.400543281010219
Epoch 50: val loss 0.725306

Epoch %d: train loss %f 51 0.46862009167671204
Epoch 51: val loss 0.730037

Epoch %d: train loss %f 52 0.43771082162857056
Epoch 52: val loss 0.723650

Epoch %d: train loss %f 53 0.44094665561403545
Epoch 53: val loss 0.719631

Epoch %d: train loss %f 54 0.37773976581437246
Epoch 54: val loss 0.704478

Epoch %d: train loss %f 55 0.46204908405031475
Epoch 55: val loss 0.709765

Epoch %d: train loss %f 56 0.40066937037876676
Epoch 56: val loss 0.703380

Epoch %d: train loss %f 57 0.39045679569244385
Epoch 57: val loss 0.723069

Epoch %d: train

...

Epoch 185: val loss 0.825498

Epoch %d: train loss %f 186 0.38335400943954784
Epoch 186: val loss 0.841419

Epoch %d: train loss %f 187 0.4032572979728381
Epoch 187: val loss 0.866345

Epoch %d: train loss %f 188 0.44759351760149
Epoch 188: val loss 0.838163

Epoch %d: train loss %f 189 0.37825920681158703
Epoch 189: val loss 0.849909

Epoch %d: train loss %f 190 0.3767115039130052
Epoch 190: val loss 0.846456

Epoch %d: train loss %f 191 0.3963802754878998
Epoch 191: val loss 0.826054

Epoch %d: train loss %f 192 0.4255279464026292
Epoch 192: val loss 0.844441

Epoch %d: train loss %f 193 0.393744391699632
Epoch 193: val loss 0.869197

Epoch %d: train loss %f 194 0.36835047230124474
Epoch 194: val loss 0.858055

Epoch %d: train loss %f 195 0.38263751069704693
Epoch 195: val loss 0.875219

Epoch %d: train loss %f 196 0.3892783572276433
Epoch 196: val loss 0.880455

Epoch %d: train loss %f 197 0.3749927381674449
Epoch 197: val loss 0.885961

Epoch %d: train loss %f 198 0.414153372248013

Epoch %d: train loss %f 95 0.44830937186876935
Epoch 95: val loss 0.739061

Epoch %d: train loss %f 96 0.48148082693417865
Epoch 96: val loss 0.734078

Epoch %d: train loss %f 97 0.45504296322663623
Epoch 97: val loss 0.726333

Epoch %d: train loss %f 98 0.4528656875093778
Epoch 98: val loss 0.722552

Epoch %d: train loss %f 99 0.46721725165843964
Epoch 99: val loss 0.728861

Epoch %d: train loss %f 100 0.44976946463187534
Epoch 100: val loss 0.743694

Epoch %d: train loss %f 101 0.4391310264666875
Epoch 101: val loss 0.733792

Epoch %d: train loss %f 102 0.5029645711183548
Epoch 102: val loss 0.741254

Epoch %d: train loss %f 103 0.43961311131715775
Epoch 103: val loss 0.743671

Epoch %d: train loss %f 104 0.4706901013851166
Epoch 104: val loss 0.744328

Epoch %d: train loss %f 105 0.49046890437602997
Epoch 105: val loss 0.732381

Epoch %d: train loss %f 106 0.4549747755130132
Epoch 106: val loss 0.724564

Epoch %d: train loss %f 107 0.43682679533958435
Epoch 107: val loss 0.728184

E

Epoch %d: train loss %f 4 0.6793391903241476
Epoch 4: val loss 0.679898

Epoch %d: train loss %f 5 0.6762271771828333
Epoch 5: val loss 0.678864

Epoch %d: train loss %f 6 0.6752138336499532
Epoch 6: val loss 0.679194

Epoch %d: train loss %f 7 0.6644738018512726
Epoch 7: val loss 0.677468

Epoch %d: train loss %f 8 0.6563408573468527
Epoch 8: val loss 0.675320

Epoch %d: train loss %f 9 0.6632959047953287
Epoch 9: val loss 0.676270

Epoch %d: train loss %f 10 0.6521390328804652
Epoch 10: val loss 0.677471

Epoch %d: train loss %f 11 0.6548417260249456
Epoch 11: val loss 0.674245

Epoch %d: train loss %f 12 0.6484570056200027
Epoch 12: val loss 0.672830

Epoch %d: train loss %f 13 0.6383401701847712
Epoch 13: val loss 0.672443

Epoch %d: train loss %f 14 0.6397804170846939
Epoch 14: val loss 0.670049

Epoch %d: train loss %f 15 0.6289492001136144
Epoch 15: val loss 0.669736

Epoch %d: train loss %f 16 0.6312970469395319
Epoch 16: val loss 0.671751

Epoch %d: train loss %f 17 0.62599678

Epoch %d: train loss %f 117 0.43579493711392087
Epoch 117: val loss 0.687453

Epoch %d: train loss %f 118 0.4493430244425933
Epoch 118: val loss 0.698183

Epoch %d: train loss %f 119 0.47042205184698105
Epoch 119: val loss 0.694187

Epoch %d: train loss %f 120 0.4246762692928314
Epoch 120: val loss 0.718383

Epoch %d: train loss %f 121 0.448782317340374
Epoch 121: val loss 0.707696

Epoch %d: train loss %f 122 0.43095844735701877
Epoch 122: val loss 0.721202

Epoch %d: train loss %f 123 0.4159177392721176
Epoch 123: val loss 0.714670

Epoch %d: train loss %f 124 0.44518420845270157
Epoch 124: val loss 0.723241

Epoch %d: train loss %f 125 0.4556265001495679
Epoch 125: val loss 0.723080

Epoch %d: train loss %f 126 0.41275156537691754
Epoch 126: val loss 0.717177

Epoch %d: train loss %f 127 0.4530874341726303
Epoch 127: val loss 0.717637

Epoch %d: train loss %f 128 0.4146236553788185
Epoch 128: val loss 0.718700

Epoch %d: train loss %f 129 0.42847610265016556
Epoch 129: val loss 0.72

Epoch 26: val loss 0.660621

Epoch %d: train loss %f 27 0.5969574842602015
Epoch 27: val loss 0.645220

Epoch %d: train loss %f 28 0.5800967924296856
Epoch 28: val loss 0.655146

Epoch %d: train loss %f 29 0.5857073329389095
Epoch 29: val loss 0.652411

Epoch %d: train loss %f 30 0.5657981503754854
Epoch 30: val loss 0.653009

Epoch %d: train loss %f 31 0.5704933013767004
Epoch 31: val loss 0.660575

Epoch %d: train loss %f 32 0.5466318149119616
Epoch 32: val loss 0.664425

Epoch %d: train loss %f 33 0.5557166989892721
Epoch 33: val loss 0.667563

Epoch %d: train loss %f 34 0.5486248359084129
Epoch 34: val loss 0.668793

Epoch %d: train loss %f 35 0.5699019096791744
Epoch 35: val loss 0.662875

Epoch %d: train loss %f 36 0.5669411290436983
Epoch 36: val loss 0.662967

Epoch %d: train loss %f 37 0.5711689554154873
Epoch 37: val loss 0.655795

Epoch %d: train loss %f 38 0.5400270074605942
Epoch 38: val loss 0.641845

Epoch %d: train loss %f 39 0.5718028154224157
Epoch 39: val loss 0.6431

This piece will save general information about training in file res.txt contating validating, test naive and test ensamble AUCS for each subject and mean AUC for all subjects

In [28]:
write_results_table(subjs_test_stats, path_to_exp=experiment_res_dir)